视频讲解地址



https://www.bilibili.com/video/BV1AE411r7ph?p=10

In [ ]:
import requests

def download(link, fpath):
    """
    下载多媒体及文件
    link： 多媒体文件链接（结尾有文件格式名）
    fpath: 存储文件的路径（结尾有文件格式名）
    """
    
    resp = requests.get(link)
    #获取到二进制数据
    binarydata = resp.content
    #以二进制形式将数据流存入fname中
    with open(fpath, 'wb') as f:
        f.write(binarydata)
        

link = 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/new/2021-05-14/601665_20210514_3.pdf'
fpath = 'data/test.pdf'
download(link, fpath)

In [ ]:
import random
import requests
from pyquery import PyQuery
import re
import time
import csv


url = 'http://www.szse.cn/api/search/content?random={rdm}'.format(rdm=random.random())

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}


formdata = {'keyword': '首次招股说明',
            'range': 'title',
            'time': 1,
            'orderby': 'score',
            'currentPage': 1,
            'pageSize': 20}


with open('深圳交易所.csv', 'w', encoding='utf-8', newline='') as csvf:
    fieldnames = ['title', 'date', 'content', 'link']
    writer = csv.DictWriter(csvf, fieldnames=fieldnames)
    writer.writeheader()
    
    resp = requests.post(url, headers=headers, data=formdata)
    totalSize = resp.json()['totalSize']
    max_page_num = int(totalSize/20)+1
    for page in range(1, max_page_num+1):
        formdata['currentPage'] = page
        resp = requests.post(url, headers=headers, data=formdata)
        for company in resp.json()['data']:
            row = dict()
            row['content'] = company['doccontent']
            row['link'] = company['docpuburl']
            row['date'] = re.findall('\d{4}-\d{2}-\d{2}', company['docpuburl'] )[0]
            row['title'] = PyQuery(company['doctitle']).text() 
            writer.writerow(row)
            print(row)  #doccontent
        time.sleep(1)
    


In [ ]:
## 下载
import requests
import pandas as pd

def download(link, fpath):
    """
    下载多媒体及文件
    link： 多媒体文件链接（结尾有文件格式名）
    fpath: 存储文件的路径（结尾有文件格式名）
    """
    
    resp = requests.get(link)
    #获取到二进制数据
    binarydata = resp.content
    #以二进制形式将数据流存入fname中
    with open(fpath, 'wb') as f:
        f.write(binarydata)
        
df = pd.read_csv('深圳交易所.csv')
for title, link in zip(df['title'], df['link']):
    fpath = '深圳/{title}.PDF'.format(title=title)
    download(link, fpath)